In [ ]:
import pandas as pd

# Carregando o arquivo CSV em um DataFrame e definindo a primeira linha após a exclusão das cinco primeiras como cabeçalho
df = pd.read_csv('extrato-rede_parceira.csv', skiprows=5, header=0)


# # Filtrar coluna TYPE para manter apenas valores iguais a ORDER
df = df[df['TYPE'] == 'ORDER']

# # Filtrar a coluna 'ORDER STATUS' para manter apenas 'COMPLETED'
df = df[df['ORDER STATUS'] == 'COMPLETED']

# # Excluir colunas não necessárias
colunas_para_excluir = ['TYPE', 'ACTION', 'ORDER ID', 'ORDER STATUS', 'E-PAYMENT (+/-)', 'CASH (+/-)', 'STATEMENT TRANSACTIONS', 'FINAL AMOUNT', 'USER', 'USER ID', 'PICKUP DATE', 'CREATED', 'COMPLETED TIME', 'ORDER CITY', 'ADDRESSES', 'NUMBER OF STOPS', 'SPECIAL REQUEST ITEM', 'ORDER CONTACT', 'DRIVER ID', 'SPECIAL REQUEST', 'LARGE PACKAGE FEE', 'PREMIUM FEE', 'COUPON AMOUNT', 'SURCHARGE DISCOUNT', 'SAVER DISCOUNT', 'CS SUBSIDY', 'ADD ON FEE', 'MULTISTOP SURCHARGE', 'VEHICLE SPECIFICATION', 'CS ADJUSTMENT', 'REFUND', 'CANCELLATION FEE', 'CANCELLATION FEE REFUND', 'SERVICE TYPE', 'ORDER PATH', 'REFUND DATE']
df = df.drop(columns=colunas_para_excluir)

# Extrair o id da coluna 'ORDER REMARK' e criar a 'ORDER APPJUSTO'
df['ORDER APPJUSTO'] = df['ORDER REMARK'].str.extract(r'#(\d+)')

# Excluir a coluna 'ORDER REMARK'
df = df.drop(columns=['ORDER REMARK'])

# Definir um dicionário de mapeamento de colunas antigas para colunas traduzidas
novo_nome_das_colunas = {
    'CREDITS (+/-)': 'TOTAL PAGO LALAMOVE',
    'DELIVERY FEE': 'TAXA DE ENTREGA',
    'STARTING PRICE': 'PREÇO INICIAL',
    'EXTRA MILEAGE PRICE': 'PREÇO KM EXTRA',
    'PRIORITY FEE': 'TAXA DE PRIORIDADE',
    'SURCHARGE FEE': 'TAXA DE SOBRECARGA'
}

# Renomear as colunas usando o dicionário
df.rename(columns=novo_nome_das_colunas, inplace=True)


# Reordenar as colunas:
colunas_ordem = ['ORDER APPJUSTO', 'DISTANCE (KM)', 'TOTAL PAGO LALAMOVE', 'TAXA DE ENTREGA', 'PREÇO INICIAL', 'PREÇO KM EXTRA', 'TAXA DE PRIORIDADE', 'TAXA DE SOBRECARGA']

df = df[colunas_ordem]

# Converte sinal para positivo para cálculo após junção com a tabela terceiriados-appjusto
df['TOTAL PAGO LALAMOVE'] = -1 * df['TOTAL PAGO LALAMOVE']



display(df)

,ORDER APPJUSTO,DISTANCE (KM),TOTAL PAGO LALAMOVE,TAXA DE ENTREGA,PREÇO INICIAL,PREÇO KM EXTRA,TAXA DE PRIORIDADE,TAXA DE SOBRECARGA
0,59391,4.35,13.42,13.42,0.0,0.00,0.00,0.0
1,59318,5.07,16.60,16.60,0.0,0.00,0.00,0.0
2,59395,5.04,17.71,0.00,6.0,4.71,4.00,3.0
3,59397,1.00,9.00,0.00,6.0,0.00,0.00,3.0
4,59403,1.69,10.00,6.00,0.0,0.00,4.00,0.0
...,...,...,...,...,...,...,...,...
1279,62612,21.57,30.00,25.03,0.0,0.00,4.97,0.0
1280,62616,2.67,12.00,6.76,0.0,0.00,5.24,0.0
1281,62606,3.10,18.00,7.15,0.0,0.00,10.85,0.0
1282,62617,3.60,11.64,7.64,0.0,0.00,4.00,0.0


In [6]:
# Carregando o segundo arquivo CSV em um DataFrame
df2 = pd.read_csv('terceirizados.csv')

# 1. Excluir valores igual a null na coluna 'TELEFONE' e suas linhas correspondentes
df2 = df2[df2['TELEFONE'].notnull()]

# # 2. Excluir as colunas 'NOME' e 'TELEFONE'
colunas_para_excluir = ['NOME', 'TELEFONE']
df2 = df2.drop(columns=colunas_para_excluir)

display(df2)


,DATA,PEDIDO,VALOR PAGO CLIENTE
0,1 de dez. de 2023,62623,25.86
1,1 de dez. de 2023,62617,10.47
2,1 de dez. de 2023,62616,10.47
3,1 de dez. de 2023,62612,46.25
4,1 de dez. de 2023,62609,10.33
...,...,...,...
1340,1 de nov. de 2023,59403,10.47
1341,1 de nov. de 2023,59401,15.95
1342,1 de nov. de 2023,59397,10.33
1343,1 de nov. de 2023,59395,16.23


In [7]:
# Exclua as linhas que contêm NaN na coluna 'ORDER APPJUSTO'
df = df.dropna(subset=['ORDER APPJUSTO'])

# # Converta 'ORDER APPJUSTO' para o tipo int
df['ORDER APPJUSTO'] = df['ORDER APPJUSTO'].astype(int)

# # Converta 'PEDIDO' para o tipo int
df2['PEDIDO'] = df2['PEDIDO'].astype(int)

# # Realize a mesclagem dos DataFrames com base nas colunas 'ORDER APPJUSTO' e 'PEDIDO'
resultado = df.merge(df2, left_on='ORDER APPJUSTO', right_on='PEDIDO', how='inner')

# # Calcula a diferença entre as colunas 'TOTAL PAGO LALAMOVE' e 'VALOR PAGO CLIENTE'
resultado['DIFERENÇA ENTRE VALORES'] = resultado['TOTAL PAGO LALAMOVE'] - resultado['VALOR PAGO CLIENTE']

# # Arredonda os valores para duas casas decimais e formata
resultado['DIFERENÇA ENTRE VALORES'] = resultado['DIFERENÇA ENTRE VALORES'].apply(lambda x: round(x, 2))
resultado['DIFERENÇA ENTRE VALORES'] = resultado['DIFERENÇA ENTRE VALORES'].apply(lambda x: f"R$ {x:.2f}".replace('.', ','))

# Exlui colunas desnecessárias
col_para_excluir = ['PEDIDO']
resultado = resultado.drop(columns=col_para_excluir)

col_ordem = ['DATA', 'ORDER APPJUSTO', 'DISTANCE (KM)', 'TAXA DE ENTREGA', 'PREÇO INICIAL', 'PREÇO KM EXTRA', 'TAXA DE PRIORIDADE', 'TAXA DE SOBRECARGA', 'TOTAL PAGO LALAMOVE', 'VALOR PAGO CLIENTE', 'DIFERENÇA ENTRE VALORES']

resultado = resultado[col_ordem]

# Salva o DataFrame em um arquivo CSV
resultado.to_csv('resultado.csv', index=False)

display(resultado)

,DATA,ORDER APPJUSTO,DISTANCE (KM),TAXA DE ENTREGA,PREÇO INICIAL,PREÇO KM EXTRA,TAXA DE PRIORIDADE,TAXA DE SOBRECARGA,TOTAL PAGO LALAMOVE,VALOR PAGO CLIENTE,DIFERENÇA ENTRE VALORES
0,1 de nov. de 2023,59391,4.35,13.42,0.0,0.00,0.00,0.0,13.42,10.47,"R$ 2,95"
1,1 de nov. de 2023,59395,5.04,0.00,6.0,4.71,4.00,3.0,17.71,16.23,"R$ 1,48"
2,1 de nov. de 2023,59397,1.00,0.00,6.0,0.00,0.00,3.0,9.00,10.33,"R$ -1,33"
3,1 de nov. de 2023,59403,1.69,6.00,0.0,0.00,4.00,0.0,10.00,10.47,"R$ -0,47"
4,1 de nov. de 2023,59401,6.63,18.20,0.0,0.00,0.00,0.0,18.20,15.95,"R$ 2,25"
...,...,...,...,...,...,...,...,...,...,...,...
1200,1 de dez. de 2023,62612,21.57,25.03,0.0,0.00,4.97,0.0,30.00,46.25,"R$ -16,25"
1201,1 de dez. de 2023,62616,2.67,6.76,0.0,0.00,5.24,0.0,12.00,10.47,"R$ 1,53"
1202,1 de dez. de 2023,62606,3.10,7.15,0.0,0.00,10.85,0.0,18.00,12.30,"R$ 5,70"
1203,1 de dez. de 2023,62617,3.60,7.64,0.0,0.00,4.00,0.0,11.64,10.47,"R$ 1,17"


In [22]:
import pandas as pd
import plotly.graph_objects as go

# Carregando os dados
resultado = pd.read_csv('resultado.csv')

# Agrupando os dados
dados_agrupados = resultado.groupby('DATA')[['VALOR PAGO CLIENTE', 'TOTAL PAGO LALAMOVE']].sum().reset_index()

# Calculando a diferença entre 'Total Pago LALAMOVE' e 'Valor Pago Cliente'
dados_agrupados['Diferença entre valores'] = dados_agrupados['TOTAL PAGO LALAMOVE'] - dados_agrupados['VALOR PAGO CLIENTE']

# Encontrar os top 3 dias com as maiores diferenças
top_dias = dados_agrupados.nlargest(3, 'Diferença entre valores')

# Alturas individuais para as anotações (ajuste o valor de y para mover as anotações para cima)
alturas = [100, 80, 90]

# Calculando as somas
soma_km_extra = resultado['PREÇO KM EXTRA'].sum()
soma_prioridade = resultado['TAXA DE PRIORIDADE'].sum()
soma_sobrecarga = resultado['TAXA DE SOBRECARGA'].sum()
soma_tres_anteriores = soma_km_extra + soma_prioridade + soma_sobrecarga
soma_total_lalamove = resultado['TOTAL PAGO LALAMOVE'].sum()
soma_total_cliente = resultado['VALOR PAGO CLIENTE'].sum()

# Calculando a soma da diferença
soma_diferenca = dados_agrupados['Diferença entre valores'].sum()

# Criando um gráfico de barras interativo com Plotly
fig = go.Figure()

# Formatação dos valores para BRL
formatar_para_BRL = lambda x: f'R$ {x:.2f}'.replace('.', ',')

fig.add_trace(go.Bar(x=dados_agrupados['DATA'], y=dados_agrupados['VALOR PAGO CLIENTE'], 
                     text=dados_agrupados['VALOR PAGO CLIENTE'].apply(formatar_para_BRL),
                     name='Valor Pago Cliente',
                     marker_color='rgb(108, 232, 134)'))

fig.add_trace(go.Bar(x=dados_agrupados['DATA'], y=dados_agrupados['TOTAL PAGO LALAMOVE'], 
                     text=dados_agrupados['TOTAL PAGO LALAMOVE'].apply(formatar_para_BRL),
                     name='Total Pago LALAMOVE',
                     marker_color='rgb(255, 103, 29)'))

# Adicionar setas para indicar as barras com as maiores diferenças
for row, altura in zip(top_dias.itertuples(), alturas):
    fig.add_annotation(
        text=f'Diferença: {formatar_para_BRL(row._4)}',  # Usamos row._4 para acessar a coluna 'Diferença entre valores'
        x=row.DATA,
        y=row._4,
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor='red',
        font=dict(size=12, color='black'),
        yshift=altura  # Altura individual para cada anotação
    )

# Personalização do layout
fig.update_xaxes(categoryorder='array', categoryarray=dados_agrupados['DATA'])
fig.update_xaxes(tickangle=45)
fig.update_layout(xaxis_title='Data', yaxis_title='Valores', legend_title='Legenda')

# Exibir as somas no topo do gráfico
text = f'Soma KM Extra: {formatar_para_BRL(soma_km_extra)}<br>'
text += f'Soma Prioridade: {formatar_para_BRL(soma_prioridade)}<br>'
text += f'Soma Sobrecarga: {formatar_para_BRL(soma_sobrecarga)}<br>'
text += f'Soma 3 Anteriores: {formatar_para_BRL(soma_tres_anteriores)}<br>'
text += f'Soma Total Pago LALAMOVE: {formatar_para_BRL(soma_total_lalamove)}<br>'
text += f'Soma Valor Pago Cliente: {formatar_para_BRL(soma_total_cliente)}'

p_n = "Valor recebido a mais do que gastamos" if soma_diferenca < 0 else "Valor que gastamos a mais do que recebemos"


fig.update_layout(title=f'Valores por Data - Soma da Diferença: {formatar_para_BRL(-1 *soma_diferenca)} ({p_n})')

fig.add_annotation(
    text=text,
    xref='paper', yref='paper',
    x=1, y=1.05,  # Ajuste a coordenada y para mover o texto para cima
    showarrow=False,
    font=dict(size=12)
)

fig.show()
